## 🤖 TFB Prediction Tutorial 2/4: Model Initialization - From Task to Architecture

In the previous tutorial, [01_data_preparation.ipynb](./01_data_preparation.ipynb), we defined our biological task, predicting transcription factor binding sites, and prepared our data accordingly. We framed it as a **multi-label sequence classification** problem. This crucial step of defining the task and data format dictates our next decision: **choosing the right model architecture**.


Welcome to the second part of our **streamlined** tutorial series. In the previous tutorial, we prepared our data with just 3 lines of code using the enhanced OmniDataset framework. Now, we will focus on **Model Initialization** with equal simplicity.

> 💡 **Learning Objectives**: Master the universal model initialization pattern, understand foundation model concepts, and leverage OmniGenBench's intelligent defaults

---

## The Power of Pre-trained Models 🚀

The core idea behind fine-tuning is to leverage **pre-trained foundation models**. These models have already learned the fundamental "language" of the genome from vast amounts of unlabeled sequence data. This pre-training endows them with powerful, general-purpose understanding of genomic patterns.

Our task is to take this general knowledge and specialize it for our specific problem: translation efficiency prediction. With the enhanced OmniGenBench framework, this process is now **effortless**.

### 🧠 Why Use Foundation Models Instead of Traditional Methods?

| Traditional Methods | Foundation Model Approach |
|---------|------------|
| 🔴 Requires hand-crafted features | 🟢 Automatically learns feature representations |
| 🔴 Relies on prior biological knowledge | 🟢 Discovers patterns from data |
| 🔴 Limited generalization ability | 🟢 Strong cross-task generalization |
| 🔴 Needs large amounts of task-specific data | 🟢 Can fine-tune with small amounts of data |

## Key Components: Model and Tokenizer 🔧

The enhanced OmniGenBench framework has revolutionized model initialization. What once required complex configuration is now **automatic**.


This tutorial will guide you through the process of selecting and initializing a model from the `OmniGenBench` framework. We will cover:

1.  **The OmniGenBench Model Zoo**: An overview of the various pre-packaged model architectures available in `OmniGenBench`.
2.  **The Principle of Model Selection**: How to choose the appropriate model based on the task you defined.
3.  **Model Architecture**: Understanding the "base model + task head" design.
4.  **Inputs and Outputs**: What the model expects as input and what it produces as output.
5.  **Practical Implementation**: Initializing a model for our TFB prediction task.

By the end of this tutorial, you will understand how to translate a biological problem into a concrete model architecture ready for training.

### 1. The OmniGenBench Model Zoo: A Task-Oriented Collection

`OmniGenBench` is not just a single model; it's a comprehensive framework that provides a variety of model architectures, each tailored for a specific type of biological task. These are often referred to as "task heads." When you use a foundation model, you are combining a powerful, pre-trained **base model** (which has learned fundamental patterns in genomic data) with a smaller, task-specific **head**.

Here is a summary of the main model classes available in `OmniGenBench` and the tasks they are designed for:

| Model Class                                       | Task Type                    | Biological Example                                       |
| ------------------------------------------------- | ---------------------------- | -------------------------------------------------------- |
| `OmniModelForSequenceClassification`              | Sequence Classification      | Classifying a promoter sequence as active or inactive.   |
| `OmniModelForMultiLabelSequenceClassification`    | Multi-Label Classification   | Predicting multiple TFB sites in one DNA sequence.       |
| `OmniModelForTokenClassification`                 | Token Classification         | Identifying splice sites at specific positions in a gene. |
| `OmniModelForSequenceRegression`                  | Sequence Regression          | Predicting the overall translation efficiency of an mRNA. |
| `OmniModelForTokenRegression`                     | Token Regression             | Predicting per-base modification status (e.g., m6A).     |
| `OmniModelForSeq2Seq`                             | Sequence-to-Sequence         | Translating a DNA sequence to a protein sequence.        |
| `OmniModelForRNADesign`                           | Sequence Generation          | Designing an RNA sequence that folds into a target shape. |
| `OmniModelForMLM` (Masked Language Model)         | Self-Supervised Pre-training | Learning general representations from unlabeled DNA.     |

This modular design is powerful because it allows you to leverage the same foundational knowledge for many different problems. You simply swap out the "head" to match your specific task.

### 2. The Principle of Model Selection: Matching Task to Architecture

The selection principle is straightforward: **match the model architecture to the machine learning task you defined in the data preparation phase.**

In our case:
-   **Biological Problem**: Predicting if any of 919 transcription factors bind to a given DNA sequence.
-   **Data Format**: A DNA sequence of length 1000.
-   **Label Format**: A binary vector of length 919, where each element indicates binding (1) or no binding (0) for a specific TF.
-   **ML Task**: Since a single sequence can have multiple TFs binding to it, this is a **Multi-Label Sequence Classification** problem.

Based on the table above, the clear choice for this task is `OmniModelForMultiLabelSequenceClassification`. This model is specifically designed to take a single sequence as input and produce multiple independent binary classifications.

### 3. Model Architecture: Base Model + Task Head

Let's visualize the architecture. At its core, our model consists of two parts:

1.  **The Base Model**: This is a large, pre-trained transformer model (like OmniGenome-186M) that has been trained on a massive amount of unlabeled genomic data. Its job is to read a DNA sequence (as a series of tokens) and convert it into a rich numerical representation (an embedding). This embedding captures the complex syntax and semantics of the genomic language.
2.  **The Task Head**: This is a smaller, simple neural network (usually just one or two linear layers) that sits on top of the base model. It takes the sequence embedding from the base model and transforms it into the desired output format for our specific task. For our TFB problem, this is a **multi-label classification head**.

Here is a diagram illustrating this concept:

```mermaid
graph TD
    subgraph "Input"
        A[DNA Sequence<br/>"GATTACA..."]
    end

    subgraph "Tokenization"
        B[Input Tokens<br/>"[CLS], G, A, T, T, A, A, [SEP]"]
    end

    subgraph "OmniModelForMultiLabelSequenceClassification"
        C(Base Model<br/>OmniGenome-186M)
        D(Classification Head<br/>Linear Layer + Sigmoid)
    end
    
    subgraph "Output"
        E[Prediction Vector<br/>"[0.9, 0.1, 0.05, ..., 0.8]"]
    end

    A --> B
    B --> C
    C -- Sequence Embedding --> D
    D --> E
```

The base model does the heavy lifting of understanding the sequence, while the head adapts that understanding to our specific predictive goal. During fine-tuning, we will update the weights of both the head and (to a lesser extent) the base model to optimize for our TFB prediction task.

### 4. Inputs and Outputs: A Look at the Data Flow

Understanding what the model expects and what it returns is critical for debugging and interpretation.

-   **Model Input**: The model expects a batch of tokenized sequences. When you use the `AccelerateTrainer` (which we will see in the next tutorial), the data loader will automatically prepare a dictionary-like object for each batch. This object contains:
    -   `input_ids`: A tensor of token IDs representing the DNA sequences.
    -   `attention_mask`: A tensor indicating which tokens are real and which are padding.
    -   `labels`: A tensor containing the ground-truth labels for each sequence.

-   **Model Output**: During training, the model returns a dictionary containing:
    -   `loss`: The calculated loss value, which the trainer uses to update the model weights.
    -   `logits`: The raw, unnormalized output scores from the final linear layer. For our task, this will be a tensor of shape `(batch_size, 919)`.

During inference (prediction), the model simply returns the `logits`. To get probabilities, we typically apply a Sigmoid function to the logits. To get binary predictions, we can then threshold these probabilities (e.g., predict 1 if probability > 0.5).

### 5. Practical Implementation: Initializing the Model

Now, let's translate this theory into code. We will perform the following steps:
1.  Import necessary libraries.
2.  Define a configuration object to hold all our parameters.
3.  Write a function to initialize the tokenizer and the `OmniModelForMultiLabelSequenceClassification` model.

First, let's set up our environment by importing the required modules.

In [ ]:
import torch
from omnigenbench import (
    OmniTokenizer, 
    OmniModelForMultiLabelSequenceClassification
)

Next, we define a configuration dictionary. This is a best practice that centralizes all important parameters, making the code cleaner and easier to modify.

In [ ]:
# Configuration - matches complete tutorial exactly
config = {
    "model_name_or_path": "yangheng/OmniGenome-52M",
    "num_labels": 919,
    "max_length": 512,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

print("⚙️ Configuration:")
print(f"  🧬 Model: {config['model_name_or_path']}")
print(f"  🏷️ Labels: {config['num_labels']} TF binding sites")
print(f"  📏 Max length: {config['max_length']} tokens")
print(f"  📱 Device: {config['device']}")

Finally, we create the function that loads our model. This function encapsulates the logic for:
-   Loading the pre-trained tokenizer.
-   Loading the pre-trained base model.
-   Initializing our chosen task-specific model, `OmniModelForMultiLabelSequenceClassification`, which wraps the base model and adds the classification head.

In [ ]:
# Load tokenizer - matches complete tutorial
print("🔄 Loading tokenizer...")
tokenizer = OmniTokenizer.from_pretrained(config["model_name_or_path"])
print(f"✅ Tokenizer loaded: {config['model_name_or_path']}")

# Load model - matches complete tutorial exactly
print("🔄 Loading model...")
model = OmniModelForMultiLabelSequenceClassification(
    config["model_name_or_path"],
    tokenizer,
    num_labels=config["num_labels"],
)

# Move model to the specified device
model.to(config["device"])

print("✅ Model and tokenizer loaded successfully.")
print(f"🎯 Model is on device: {next(model.parameters()).device}")

# Show model statistics
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Model parameters: {total_params:,} total, {trainable_params:,} trainable")
print(f"🧬 Ready for {config['num_labels']}-label TF binding prediction")

### Summary and Next Steps

Congratulations! You have successfully initialized a powerful Genomic Foundation Model tailored for your specific biological task.

To recap, we have:
-   Surveyed the different model architectures available in `OmniGenBench`.
-   Learned the principle of matching the model architecture to the ML task.
-   Understood the "base model + task head" design.
-   Initialized the `OmniModelForMultiLabelSequenceClassification` model and its tokenizer.

We now have the two key components ready: a prepared dataset and an initialized model. The next logical step is to bring them together and train the model to make accurate predictions.

In the next tutorial, **[3/4: Model Training](./03_model_training.ipynb)**, we will take this model and the data we prepared in the first tutorial and begin the fine-tuning process.